<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-T/blob/master/DataAlgo_T(007)_Longest_Path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Path Problem in Graph

A path from Vertex A to Vertex B is a seris of vercies and the associated edges between the vertices. In direct graph, the direction should be the same from the source vertex A to the destination vertex B.

A path may visit the same vertex several times, and it may include cycle(s).

However, we usually assume that a vertex appers in a path at most once (and no cycles). It is called a simple path.

In this section, we lern two typical path problems -- longest path and shortest path.

# 4.1. Longest Path Problem

Learn the algorithm to find the longest path in a given graph.

**Reminder**  
On github, rendering might not be in good shape.  
To see the expected layout, open this page in Google Colaboratory.
To run one specific code cell in colab, click the icon on the left part or just type Ctrl + Enter.  


# Preparation

Connect the Jupyter environment and invoke a runtime. 
Mount your Google Drive by the procedure below.  
Change directory to the mounted point and make it as the working folder.  
By then, files are preserved even after you terminate the runtime environment.

In [ ]:
!echo "Mounting your Google Drive"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "Make a working folder and chnage directory to it"
%cd /content/drive/My\ Drive
%mkdir -p DataAlgo-T/007
%cd       DataAlgo-T/007
!ls

Here we consider weighted graphs that are directed and that do not have cycles.  
Weights are positive value, and they are attahed to edges. 

Consider Graph G5 as an example.  
8 vertices, directed, no cycles.  
(Graph G4 can be treated as a weighted graph with value 1 for every edge.)  
In this example, weights are integer, but any positive value is allowed.

![da2020-graph5](https://user-images.githubusercontent.com/45651568/84226060-bfe2cd00-ab1b-11ea-8493-9b077d00e569.jpg)

In [ ]:
%%writefile graph5.h
// 8 nodes, directed, no cycle.
#define N 8
int edge[N][N] = {
//   0 1 2 3 4 5 6 7
	{0,4,0,0,2,0,0,0}, // 0
	{0,0,0,0,0,0,1,0}, // 1
	{0,0,0,3,0,0,3,0}, // 2
	{0,0,0,0,0,0,0,5}, // 3
	{0,0,0,0,0,2,0,0}, // 4
	{0,0,0,0,0,0,0,0}, // 5
	{0,0,0,0,0,4,0,1}, // 6
	{0,0,0,0,0,0,0,0}  // 7
};

# Algorithm to solve the longest path problem

**Explanation**

The longest path problem is to find a path of maximum length in a given graph.  
The path should not include the same vertex twice. 
Length of a path is defined by the sum of the edge weights in the path.  

This means the longest path should be found per graph.

A weight of an edge between vertex A and vertex B is denoted edge(A,B).  

Suppose we think of a vertex K in a given graph.  
Let's say that the maximum length of the paths that ends at K is denoted by W(K).
Then, the definition of the problem is to find the vertex K_max and its W(K_max) where W(K_max) is the longest for all the vertices in the graph

Here, we assume that we are processing the vertex T.
We also assume all the W(S) have been obtained for all the vertices S that are located at source (upper) side of Vertex T.  
The source side can be determined by executing the topological sorting in advance.  

Then, W(D) should be calculated by:
> W(D) = max ( W(S) + edge(S,D) )   
> for any S in the source side of D  

After we obtain all the W(D) for any vertice D in the graph, finally we find the largerst W(K_max) by just scanning all the W(D).

Therefore, the algorithm to solve the longest path problem should be three-folded.

1. Obtain the vertex order list of topological sorting on a given graph.
2. From the source side of the list, take up a vertex D, and calculate W(D) towards the destination side.
3. Find the maximum length W(K_max) among all the vertices.

**Implementation**

As for the topological sorting, we reuse the program in the previous section.  
FYI, the longest path of Graph G5 weighs 9.  
Longest Path Value = 9  
5 <- 6 <- 1 <- 0 <- Go

# C Program to solve the longest path problem

**Purpose**

Obtain the weight of the longest path and its vertex list in a given direct, acyclic graph.  

**Explanation**

The program consists of three parts.

1. Topological sorting
2. Calculating W(K) for all the vertices 
3. Finding the maximum value of W()

**Program**

The program has three corresponding functions. 

**Remarks**

The third part could be implemented in the second part if you wish.
The program has some extra variables and statements to show the vertex list of the longest path.

In [ ]:
%%writefile longestpath_E.c
// Topological Sorting
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include "graph5.h"

// Structured variable for vertex
typedef struct {
	int W;   //  The largest cost of the paths that ends at this vertex.
	int src; //  The source (adjacent) vertex. It is the last part of the longest path to this vertex.
} ppath; 


int tsort[N];  // Vertex list of the topological sorting result
int num_tsort = 0; // Number of vertices in the tsort[]
ppath path[N]; // Memory to remember the longest path

int listed[N];  // 0 Not yet, 1 listed

// Step 1: Topological sorting / reuse of DFS main bod by recursive calls
void dfs(int CurrentRoom){
    int RoomToCheck;

	// It is recognized as being listed.
	listed[CurrentRoom] = 1;

    // Choose next
	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++) {
        if (edge[CurrentRoom][RoomToCheck] != 0 && listed[RoomToCheck] == 0) 
			dfs(RoomToCheck);
	}

	// Topological sorting
	printf("Listing: %d\n", CurrentRoom);

	// And record the vertex list
	tsort[num_tsort] = CurrentRoom;
	num_tsort++;

	return;
}

// Step 2: Calculate W() from the source side to the destination side
void findlongest (void) {
	int p; // The index in the vertex list tsort[]
	int d; // Vertex ID number to process

	// Cleaning up the arrays
	{
		int i;
		for (i = 0; i < N; i++) {
			path[i].W = 0; // 0 means no path to this vertex
			path[i].src = -1; // Vertex number -1 indicates no source-side (adjacent) vertex 
		}
	}
	
	// Take up a vertex from the end of tsort[] (i.e. starting from the source side in the vertex list)
	for (p = N-1; p >= 0; p--) {
		int s;
		d = tsort[p];
		for (s = 0; s < N; s++) {
			if (edge[s][d] > 0 && path[s].W + edge[s][d] > path[d].W) {
				path[d].W = path[s].W + edge[s][d];
				path[d].src = s;
			}
		}
	}

	// Show the result of this function
	printf("Node  : ");
	for (d = 0; d < N; d++) 
		printf("%3d ", d);
	printf("\n");
	printf("Weight: ");
	for (d = 0; d < N; d++) 
		printf("%3d ", path[d].W);
	printf("\n");
	printf("Src   : ");
	for (d = 0; d < N; d++) 
		printf("%3d ", path[d].src);
	printf("\n");

	return;
}

// Step 3: Find the maximum weight (of the longest path path) 
void findfinalsolution (void) {
  	int Kmax; // Ending vertex of the longest path
	int i;

	// Tentative vertex candidate (of longest path)
	// Set Vertex 0 tentatively
	Kmax = 0;

	// find the actual ending vertex of the longest path
	for (i = 1; i < N; i++) {
		if (path[i].W > path[Kmax].W)
			Kmax = i;
	}

	// Show the longest path
	printf("Longest Path Value = %d\n", path[Kmax].W);
	for (i = Kmax; i >= 0; i = path[i].src)
		printf("%d <- ", i);
	printf("Go\n");
}

// Main function
int main(int argc, char *argv[]){
	int StartingNode;
	int i;

	printf("Step 1. Topological sorting.\n");
	for (StartingNode = 0; StartingNode < N; StartingNode++) {
		if (listed[StartingNode] == 0) {
			dfs(StartingNode);
		}
	}
	printf("  Result of Topological sorting: ");
	for (i = 0; i < N; i++)
		printf("%d ", tsort[i]);
	printf("\n");

	printf("Step 2. Calculating W().\n");
	findlongest();

	printf("Step 3. Find the longest path.\n");
	findfinalsolution();

  return 0;
}

Compile it and check no errors.

In [ ]:
!gcc -Wall -o  longestpath_E longestpath_E.c

Run the program.

In [ ]:
!./longestpath_E

# Problems

1. Vertex list of the longest path  
longestpath_E.c program includes a trick to show the longest path. Point out the corresponding part of the trick and explain why it can show the vertex list (such as "5 <- 6 <- 1 <- 0 <- Go").
> See the ppath structure. ppath.src works as if it is a linked list.

2. Computation amount  
Discuss the computation amount of time and space of longestpath_E.c program.
> Time: O(N^2)，Space: O(N^2)

3. With cycles  
In this section, we assume the graph does not have cycles. By then, the given program can correctly show the answer. However, the problem definition of the longest path could allow cycles in the graph.  Investigate the algorithm that can handle directed graph with cycles (directed, cycle graphs). You are encouraged to survey it in libraries and/or the internet.
> Omitted. Actually this is NP-hard and the algorithm should be bruteforce.



#**Course Info**

https://github.com/kameda-yoshinari/DataAlgo-T  
Course: Data structure and algorithm  
Department of Engineering Systems, University of Tsukuba,Japan.  
Author: KAMEDA, Yoshinari  
2020.05.19. -  